# My First page



In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.7.0


Load and prepare the MNIST dataset. Convert the sample data from integers to floating-point numbers

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


Build a tf.keras.Sequential model by stacking layers.

In [10]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of logits or log-odds scores, one for each class.

In [11]:
predictions = model(x_train[:1]).numpy()
predictions


array([[ 0.08814757,  0.22771226, -0.2139431 , -0.09877378, -0.05175377,
        -0.14823617,  0.09557451,  0.04021232, -0.16973788, -0.4195137 ]],
      dtype=float32)

In [15]:
tf.nn.softmax(predictions).numpy()

array([[0.05361787, 0.11343007, 0.0716927 , 0.09281843, 0.08171287,
        0.22491336, 0.09245123, 0.09526451, 0.07748263, 0.09661636]],
      dtype=float32)

Define a loss function for training using losses.SparseCategoricalCrossentropy, which takes a vector of logits and a True index and returns a scalar loss for each example.


In [16]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [17]:
loss_fn(y_train[:1], predictions).numpy()

1.49204

Before you start training, configure and compile the model using Keras Model.compile. Set the optimizer class to adam, set the loss to the loss_fn function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy.


In [19]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Use the Model.fit method to adjust your model parameters and minimize the loss:

In [20]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2934 - accuracy: 0.9141
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1438 - accuracy: 0.9565
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1066 - accuracy: 0.9679
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0877 - accuracy: 0.9727
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0738 - accuracy: 0.9765


The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [21]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0732 - accuracy: 0.9784 - 530ms/epoch - 2ms/step


[0.07317705452442169, 0.9783999919891357]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [23]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [24]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.03325712e-09, 9.98350629e-07, 6.13833936e-06, 3.60014295e-04,
        9.72620179e-11, 3.58614436e-08, 6.52129904e-15, 9.99631166e-01,
        1.51283544e-07, 1.40489419e-06],
       [6.64711379e-08, 4.65822013e-05, 9.99853849e-01, 9.12261385e-05,
        4.19303252e-16, 2.25550343e-06, 6.26108161e-08, 2.51551465e-13,
        5.79218931e-06, 1.55690489e-12],
       [1.61504673e-07, 9.99071717e-01, 3.12125339e-05, 1.03848924e-05,
        5.92496253e-05, 1.35625291e-04, 9.53838753e-06, 3.89103836e-04,
        2.92271958e-04, 7.54945404e-07],
       [9.99815881e-01, 5.45249401e-10, 7.87735189e-05, 1.91137246e-07,
        7.26477012e-08, 1.37068582e-06, 8.83414614e-05, 1.20853465e-05,
        1.54206134e-07, 3.09295024e-06],
       [1.97428708e-05, 1.81751272e-08, 3.58758302e-06, 2.15697719e-06,
        9.99023676e-01, 7.27682755e-06, 9.67287342e-05, 2.42272756e-04,
        4.54856172e-06, 5.99984662e-04]], dtype=float32)>